# Import Libraries

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
import sys
import cv2
import random
import datetime
import neptune
import tempfile
import numpy as np
import pandas as pd

from imutils import paths

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as prep_input_mobilenetv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, LearningRateScheduler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


if '../../../notebooks/' not in sys.path:
    sys.path.append('../../../notebooks/')

import utils.constants as cts
import utils.draw_utils as dr

from models.oface_mouth_model import OpenfaceMouth

from data_loaders.data_loader import DLName

from net_data_loaders.net_data_loader import NetDataLoader

# from gt_loaders.gen_gt import Eval
# from gt_loaders.fvc_gt import FVC_GTLoader
# from gt_loaders.pybossa_gt import PybossaGTLoader

## Restrict GPU Memory Growth

In [3]:
## restrict memory growth -------------------

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
    tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
    raise Exception("Invalid device or cannot modify virtual devices once initialized.")

## restrict memory growth -------------------

# Start Experiment Manager

In [4]:
print('Starting Neptune')
neptune.init('guilhermemg/icao-nets-training')    
    
def log_data(logs):
    neptune.log_metric('epoch_accuracy', logs['accuracy'])
    neptune.log_metric('epoch_val_accuracy', logs['val_accuracy'])
    neptune.log_metric('epoch_loss', logs['loss'])    
    neptune.log_metric('epoch_val_loss', logs['val_loss'])    

    
def lr_scheduler(epoch):
    if epoch < 10:
        new_lr = PARAMS['learning_rate']
    else:
        new_lr = PARAMS['learning_rate'] * np.exp(0.1 * ((epoch//50)*50 - epoch))

    neptune.log_metric('learning_rate', new_lr)
    return new_lr

Starting Neptune


# Load Data

In [5]:
m = OpenfaceMouth()
req = cts.ICAO_REQ.MOUTH
dl_names = [DLName.FVC_PYBOSSA]
print(f'DL names: {dl_names}')

print('Loading data')
netDataLoader = NetDataLoader(m, req, dl_names, True)
in_data = netDataLoader.load_data()
# in_data = pd.read_csv(cts.LABELS_FQA_SCORES_DATA_PATH)
# in_data = in_data[in_data.fqa_score >= 0.5]
# in_data = in_data.astype({'comp':'str'})
print('Data loaded')

Loading data
Data loaded


# Data Generators

In [6]:
TRAIN_PROP = 0.8
VALID_PROP = 0.1
TEST_PROP = 0.1
SEED = 42

print(f'N: {len(in_data)}')

INIT_LR = 1e-4
EPOCHS = 30
BS = 32
SHUFFLE = True
DROPOUT = 0.5
# EARLY_STOPPING = 10
OPTIMIZER = 'Adam'
DENSE_UNITS = 128

print('Starting data generators')
train_valid_df = in_data.sample(frac=TRAIN_PROP+VALID_PROP, random_state=SEED)
test_df = in_data[~in_data.img_name.isin(train_valid_df.img_name)]

print('Starting data generators')
datagen = ImageDataGenerator(preprocessing_function=prep_input_mobilenetv2, 
                             validation_split=0.2)

train_gen = datagen.flow_from_dataframe(train_valid_df, 
                                        x_col="img_name", 
                                        y_col="comp",
                                        target_size=(224, 224),
                                        class_mode="binary",
                                        batch_size=BS, 
                                        subset='training')

validation_gen = datagen.flow_from_dataframe(train_valid_df,
                                            x_col="img_name", 
                                            y_col="comp",
                                            target_size=(224, 224),
                                            class_mode="binary",
                                            batch_size=BS, 
                                            subset='validation')

test_gen = datagen.flow_from_dataframe(test_df,
                                       x_col="img_name", 
                                       y_col="comp",
                                       target_size=(224, 224),
                                       class_mode="binary",
                                       batch_size=BS)

print(f'TOTAL: {train_gen.n + validation_gen.n + test_gen.n}')

N: 9639
Starting data generators
Starting data generators
Found 6940 validated image filenames belonging to 2 classes.
Found 1735 validated image filenames belonging to 2 classes.
Found 964 validated image filenames belonging to 2 classes.
TOTAL: 9639


# Create Experiment

In [10]:
# Define parameters
PARAMS = {'batch_size': BS,
          'n_epochs': EPOCHS,
          'shuffle': SHUFFLE,
          'dense_units': DENSE_UNITS,
          'learning_rate': INIT_LR,
          'optimizer': OPTIMIZER,
          'dropout': DROPOUT,
#           'early_stopping': EARLY_STOPPING,
          'train_prop': TRAIN_PROP,
          'validation_prop': VALID_PROP,
          'test_prop': TEST_PROP,
          'n_train': train_gen.n,
          'n_validation': validation_gen.n,
          'n_test': test_gen.n,
          'seed': SEED}


print('Creating experiment')
neptune.create_experiment(name='train_mobilenetv2',
                          params=PARAMS,
                          properties={
                                      'dl_names': str([dl_name.value for dl_name in dl_names]),
                                      'dl_aligned': True,
                                      'icao_req': req.value,
                                      'tagger_model': m.get_model_name().value},
                          description='Changing learning rate scheduler function to improve results from experiment IC-19',
                          tags=['mobilenetv2'],
                          upload_source_files=['train_mobilenetv2.py']
                         )

Creating experiment
https://ui.neptune.ai/guilhermemg/icao-nets-training/e/IC-20


Experiment(IC-20)

# Training Model

In [11]:
print('Training network')

baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3),
	input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(PARAMS['dense_units'], activation="relu")(headModel)
headModel = Dropout(PARAMS['dropout'])(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

# compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Log model summary
model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))

# train the head of the network
H = model.fit(
        train_gen,
        steps_per_epoch=train_gen.n // BS,
        validation_data=validation_gen,
        validation_steps=validation_gen.n // BS,
        epochs=EPOCHS,
        callbacks=[LambdaCallback(on_epoch_end = lambda epoch, logs: log_data(logs)),
#                    EarlyStopping(patience=PARAMS['early_stopping'], monitor='accuracy', restore_best_weights=True),
                   LearningRateScheduler(lr_scheduler)
                  ])

Training network
Epoch 1/30
216/216 [==============================] - 184s 853ms/step - loss: 0.7590 - accuracy: 0.4912 - val_loss: 0.6965 - val_accuracy: 0.5122
Epoch 2/30
216/216 [==============================] - 15s 68ms/step - loss: 0.6974 - accuracy: 0.5083 - val_loss: 0.6941 - val_accuracy: 0.5104
Epoch 3/30
216/216 [==============================] - 15s 69ms/step - loss: 0.6945 - accuracy: 0.5151 - val_loss: 0.6936 - val_accuracy: 0.5174
Epoch 4/30
216/216 [==============================] - 15s 70ms/step - loss: 0.6938 - accuracy: 0.5107 - val_loss: 0.6934 - val_accuracy: 0.5064
Epoch 5/30
216/216 [==============================] - 15s 69ms/step - loss: 0.6935 - accuracy: 0.4965 - val_loss: 0.6933 - val_accuracy: 0.4971
Epoch 6/30
216/216 [==============================] - 15s 68ms/step - loss: 0.6934 - accuracy: 0.4938 - val_loss: 0.6933 - val_accuracy: 0.4948
Epoch 7/30
216/216 [==============================] - 15s 68ms/step - loss: 0.6933 - accuracy: 0.4925 - val_loss: 0.6

# Saving Trained Model

# Test Trained Model

In [13]:
print("Testing Trained Model")

# make predictions on the testing set
predIdxs = model.predict(test_gen, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_gen.labels, predIdxs, target_names=['NON_COMP','COMP']))  

Testing Trained Model
              precision    recall  f1-score   support

    NON_COMP       0.57      0.27      0.36       562
        COMP       0.41      0.71      0.52       402

    accuracy                           0.45       964
   macro avg       0.49      0.49      0.44       964
weighted avg       0.50      0.45      0.43       964



# Evaluate Model

In [14]:
print('Evaluating model')
eval_metrics = model.evaluate(test_gen, verbose=0)
for j, metric in enumerate(eval_metrics):
    neptune.log_metric('eval_' + model.metrics_names[j], metric)

Evaluating model


# Finishing Experiment Manager

In [15]:
print('Finishing Neptune')
neptune.stop()

Finishing Neptune
